## Cleaning the Data
-----

### Setup:
---

In [334]:
# Import dependencies
import pandas as pd
import datetime as dt

In [335]:
# Grab the CSV
athelete_stats_df = pd.read_csv("initial_unclean_csv/unclean_data.csv", encoding="utf8")

In [336]:
# Display as dataframe
athelete_stats_df.head()

,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,clean_sheets,goals_against,photo_url,height,weight,birthdate,nationality
0,Sophia Smith,Portland Thorns FC,nwsl,Forward,9.0,11,5,2,0,75,39,0,2,16,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 6""",121 lbs,2000-08-10T07:00Z,USA
1,Sophie Schmidt,Houston Dash,nwsl,Midfielder,13.0,0,0,3,0,17,3,0,8,15,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 8""",157 lbs,1988-06-28T07:00Z,Canada
2,Christine Sinclair,Portland Thorns FC,nwsl,Forward,12.0,3,1,0,0,27,10,0,1,12,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 9""",148 lbs,1983-06-12T07:00Z,Canada
3,Abby Erceg,Racing Louisville FC,nwsl,Defender,20.0,2,0,1,0,15,6,0,6,22,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 10""",150 lbs,1989-11-20T08:00Z,New Zealand
4,Nadia Nadim,Racing Louisville FC,nwsl,Forward,10.0,1,0,0,0,14,5,0,0,5,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 9""",NaN,1988-01-02T08:00Z,Denmark


### Cleaning:
---

#### Convert Height

In [337]:
# Split current heights
split_heights = athelete_stats_df["height"]
split_heights = split_heights.str.split("'", n=1, expand=True)

In [338]:
# Remove everthing but the numbers from the feet
# Convert to float for math
heights_feet = split_heights[0].astype("float")
heights_feet.head(5)

0    5.0
1    5.0
2    5.0
3    5.0
4    5.0
Name: 0, dtype: float64

In [339]:
# Remove everything but the nubmers from the inches
heights_inches = split_heights[1]

split_heights_inches = heights_inches.str.split('"', n=1, expand=True)

# Convert to float for math
split_heights_inches = split_heights_inches[0].astype("float")
heights_inches = split_heights_inches
heights_inches.head(5)

0     6.0
1     8.0
2     9.0
3    10.0
4     9.0
Name: 0, dtype: float64

In [340]:
# Converting inches to feet
inches_in_feet = []
for measurement in heights_inches:
    cur_feet = measurement/12
    inches_in_feet.append(cur_feet)
   

In [341]:
# Create dataframe with new heights colums, both in feet
split_heights_in_ft = {
    "heights_ft1": heights_feet,
    "heights_ft2": inches_in_feet
}
split_heights_in_ft_df = pd.DataFrame(split_heights_in_ft)
split_heights_in_ft_df.head(5)

,heights_ft1,heights_ft2
0,5.0,0.500000
1,5.0,0.666667
2,5.0,0.750000
3,5.0,0.833333
4,5.0,0.750000


In [342]:
# Adding feet and converted inches column to get new total feet measurments column with decimals
new_height = split_heights_in_ft_df["heights_ft1"] + split_heights_in_ft_df["heights_ft2"]

# Round the combined height
new_height = round(new_height,2)

In [343]:
# Display cleaned data
new_height.head(5)

0    5.50
1    5.67
2    5.75
3    5.83
4    5.75
dtype: float64

#### Convert Weight

In [344]:
# Grab uncleaned weight
uncleaned_weight = athelete_stats_df["weight"]
uncleaned_weight.head(5)

0    121 lbs
1    157 lbs
2    148 lbs
3    150 lbs
4        NaN
Name: weight, dtype: object

In [345]:
# Split weights
split_weights = uncleaned_weight.str.split(' ', n=1, expand=True)
split_weights.head(5)

,0,1
0,121,lbs
1,157,lbs
2,148,lbs
3,150,lbs
4,NaN,NaN


In [346]:
# Display cleaned data
new_weight = split_weights[0]
new_weight.head(5)

0    121
1    157
2    148
3    150
4    NaN
Name: 0, dtype: object

#### Convert Dates

In [347]:
# Split Current Dates
split_birthdates = athelete_stats_df["birthdate"]
split_birthdates = split_birthdates.str.split("T", n=1, expand=True)

In [348]:
# Select only the part we want to keep
split_birthdates = split_birthdates [0]
split_birthdates.head(5)

0    2000-08-10
1    1988-06-28
2    1983-06-12
3    1989-11-20
4    1988-01-02
Name: 0, dtype: object

In [349]:
# Convert the dates to datetime
split_birthdates = pd.to_datetime(split_birthdates)
split_birthdates.head(5)

0   2000-08-10
1   1988-06-28
2   1983-06-12
3   1989-11-20
4   1988-01-02
Name: 0, dtype: datetime64[ns]

In [350]:
# Check dates converted to timestamps
type (split_birthdates[0])

pandas._libs.tslibs.timestamps.Timestamp

In [351]:
# Display cleaned data
new_birthdates = split_birthdates
new_birthdates.head(5)

0   2000-08-10
1   1988-06-28
2   1983-06-12
3   1989-11-20
4   1988-01-02
Name: 0, dtype: datetime64[ns]

#### Remove "Goals Against" and "Clean Sheets" for Non-Goalies

In [352]:
# Remove goals against and clean sheets for non-goalies
new_goals_against = []
new_clean_sheets = []

for index, row in athelete_stats_df.iterrows():
    cur_position = row["position"]
    cur_goals_against = row["goals_against"]
    cur_clean_sheets = row["clean_sheets"]
    
    if cur_position == "Goalkeeper":
        new_goals_against.append(cur_goals_against)
        new_clean_sheets.append(cur_clean_sheets)
    else:
        new_goals_against.append(0)
        new_clean_sheets.append(0)


#### Creating Cleaned Dataframe

In [353]:
# Remove unclean sections from dataframe
athelete_stats_df = athelete_stats_df.drop(["birthdate"], axis=1)
athelete_stats_df = athelete_stats_df.drop(["height"], axis=1)
athelete_stats_df = athelete_stats_df.drop(["weight"], axis=1)
athelete_stats_df = athelete_stats_df.drop(["goals_against"], axis=1)
athelete_stats_df = athelete_stats_df.drop(["clean_sheets"], axis=1)
athelete_stats_df.head()

,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,photo_url,nationality
0,Sophia Smith,Portland Thorns FC,nwsl,Forward,9.0,11,5,2,0,75,39,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,USA
1,Sophie Schmidt,Houston Dash,nwsl,Midfielder,13.0,0,0,3,0,17,3,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,Canada
2,Christine Sinclair,Portland Thorns FC,nwsl,Forward,12.0,3,1,0,0,27,10,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,Canada
3,Abby Erceg,Racing Louisville FC,nwsl,Defender,20.0,2,0,1,0,15,6,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,New Zealand
4,Nadia Nadim,Racing Louisville FC,nwsl,Forward,10.0,1,0,0,0,14,5,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,Denmark


In [354]:
# Add clean sections to dataframe
athelete_stats_df["birthdate"] = new_birthdates
athelete_stats_df["height_(ft)"] = new_height
athelete_stats_df["weight_(lbs)"] = new_weight
athelete_stats_df["goals_against"] = new_goals_against
athelete_stats_df["clean_sheets"] = new_clean_sheets
athelete_stats_df.head()

,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,photo_url,nationality,birthdate,height_(ft),weight_(lbs),goals_against,clean_sheets
0,Sophia Smith,Portland Thorns FC,nwsl,Forward,9.0,11,5,2,0,75,39,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,USA,2000-08-10,5.50,121,0,0
1,Sophie Schmidt,Houston Dash,nwsl,Midfielder,13.0,0,0,3,0,17,3,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,Canada,1988-06-28,5.67,157,0,0
2,Christine Sinclair,Portland Thorns FC,nwsl,Forward,12.0,3,1,0,0,27,10,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,Canada,1983-06-12,5.75,148,0,0
3,Abby Erceg,Racing Louisville FC,nwsl,Defender,20.0,2,0,1,0,15,6,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,New Zealand,1989-11-20,5.83,150,0,0
4,Nadia Nadim,Racing Louisville FC,nwsl,Forward,10.0,1,0,0,0,14,5,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,Denmark,1988-01-02,5.75,NaN,0,0


In [355]:
# Reorder dataframe
athelete_stats_df = athelete_stats_df[["player_name",
                                       "team",
                                       "league",
                                       "position",
                                       "jersey_number",
                                       "goals",
                                       "assists",
                                       "yellow_cards",
                                       "red_cards",
                                       "shots",
                                       "shots_on_goal",
                                       "saves",
                                       "clean_sheets",
                                       "goals_against",
                                       "photo_url",
                                       "height_(ft)",
                                       "weight_(lbs)",
                                       "birthdate",
                                       "nationality"
                                       ]]
athelete_stats_df.head()

,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,clean_sheets,goals_against,photo_url,height_(ft),weight_(lbs),birthdate,nationality
0,Sophia Smith,Portland Thorns FC,nwsl,Forward,9.0,11,5,2,0,75,39,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.50,121,2000-08-10,USA
1,Sophie Schmidt,Houston Dash,nwsl,Midfielder,13.0,0,0,3,0,17,3,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.67,157,1988-06-28,Canada
2,Christine Sinclair,Portland Thorns FC,nwsl,Forward,12.0,3,1,0,0,27,10,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.75,148,1983-06-12,Canada
3,Abby Erceg,Racing Louisville FC,nwsl,Defender,20.0,2,0,1,0,15,6,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.83,150,1989-11-20,New Zealand
4,Nadia Nadim,Racing Louisville FC,nwsl,Forward,10.0,1,0,0,0,14,5,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.75,NaN,1988-01-02,Denmark


### Creating Dataframes for SQL Tables:
---

#### Creating Leagues Dataframe

In [356]:
leagues = athelete_stats_df["league"].unique()
leagues_df = pd.DataFrame(leagues)
leagues_df = leagues_df.reset_index()
leagues_df = leagues_df.rename(columns={"index":"league_id", 0:"league"})
leagues_df["league_id"] = leagues_df["league_id"]+1
leagues_df["league_id"] = ["lg" + str(league_id) for league_id in leagues_df["league_id"]]
leagues_df

,league_id,league
0,lg1,nwsl


#### Creating Teams Dataframe

In [357]:
teams = athelete_stats_df["team"].unique()
teams_df = pd.DataFrame(teams)
teams_df = teams_df.reset_index()
teams_df = teams_df.rename(columns={"index":"team_id", 0:"team"})
teams_df["team_id"] = teams_df["team_id"]+1
teams_df["team_id"] = ["tm" + str(team_id) for team_id in teams_df["team_id"]]
teams_df.head(5)

,team_id,team
0,tm1,Portland Thorns FC
1,tm2,Houston Dash
2,tm3,Racing Louisville FC
3,tm4,Kansas City Current
4,tm5,San Diego Wave FC


In [358]:
# Add league Id's to teams dataframe
league_team_df = athelete_stats_df[["league","team"]]
league_team_df=league_team_df["team"].unique()
league_team_df

array(['Portland Thorns FC', 'Houston Dash', 'Racing Louisville FC',
       'Kansas City Current', 'San Diego Wave FC', 'Orlando Pride',
       'OL Reign', 'NJ/NY Gotham FC', 'Washington Spirit',
       'Angel City FC', 'Chicago Red Stars', 'North Carolina Courage'],
      dtype=object)

#### Creating Positions Dataframe

In [359]:
positions = athelete_stats_df["position"].unique()
positions_df = pd.DataFrame(positions)
positions_df = positions_df.reset_index()
positions_df = positions_df.rename(columns={"index":"position_id", 0:"position"})
positions_df["position_id"] = positions_df["position_id"]+1
positions_df["position_id"] = ["pos" + str(position_id) for position_id in positions_df["position_id"]]
positions_df

,position_id,position
0,pos1,Forward
1,pos2,Midfielder
2,pos3,Defender
3,pos4,Goalkeeper


#### Creating Nationalities Dataframe

In [360]:
nationalities = athelete_stats_df["nationality"].unique()
nationalities_df = pd.DataFrame(nationalities)
nationalities_df = nationalities_df.reset_index()
nationalities_df = nationalities_df.rename(columns={"index":"nationality_id", 0:"nationality"})
nationalities_df["nationality_id"] = nationalities_df["nationality_id"]+1
nationalities_df["nationality_id"] = ["nat" + str(nationality_id) for nationality_id in nationalities_df["nationality_id"]]
nationalities_df.head(5)

,nationality_id,nationality
0,nat1,USA
1,nat2,Canada
2,nat3,New Zealand
3,nat4,Denmark
4,nat5,Mexico


#### Creating Player Data Dataframe

In [361]:
# Add player ID
player_data_df = athelete_stats_df
player_data_df = player_data_df.reset_index()
player_data_df = player_data_df.rename(columns={"index":"player_id"})
player_data_df["player_id"] = player_data_df["player_id"]+1
player_data_df["player_id"] = ["pl" + str(player_id) for player_id in player_data_df["player_id"]]
player_data_df.head(5)

,player_id,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,clean_sheets,goals_against,photo_url,height_(ft),weight_(lbs),birthdate,nationality
0,pl1,Sophia Smith,Portland Thorns FC,nwsl,Forward,9.0,11,5,2,0,75,39,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.50,121,2000-08-10,USA
1,pl2,Sophie Schmidt,Houston Dash,nwsl,Midfielder,13.0,0,0,3,0,17,3,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.67,157,1988-06-28,Canada
2,pl3,Christine Sinclair,Portland Thorns FC,nwsl,Forward,12.0,3,1,0,0,27,10,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.75,148,1983-06-12,Canada
3,pl4,Abby Erceg,Racing Louisville FC,nwsl,Defender,20.0,2,0,1,0,15,6,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.83,150,1989-11-20,New Zealand
4,pl5,Nadia Nadim,Racing Louisville FC,nwsl,Forward,10.0,1,0,0,0,14,5,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.75,NaN,1988-01-02,Denmark


In [362]:
# Add ID columns
player_data_df = pd.merge(player_data_df,teams_df, on=["team"], how="inner")
player_data_df = pd.merge(player_data_df,leagues_df, on=["league"], how="inner")
player_data_df = pd.merge(player_data_df,positions_df, on=["position"], how="inner")
player_data_df = pd.merge(player_data_df,nationalities_df, on=["nationality"], how="inner")
player_data_df

,player_id,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,...,goals_against,photo_url,height_(ft),weight_(lbs),birthdate,nationality,team_id,league_id,position_id,nationality_id
0,pl1,Sophia Smith,Portland Thorns FC,nwsl,Forward,9.0,11,5,2,0,...,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.50,121,2000-08-10,USA,tm1,lg1,pos1,nat1
1,pl157,Hannah Betfort,Portland Thorns FC,nwsl,Forward,33.0,2,0,0,0,...,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.83,NaN,1999-01-04,USA,tm1,lg1,pos1,nat1
2,pl267,Izzy D'Aquila,Portland Thorns FC,nwsl,Forward,24.0,0,0,0,0,...,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.58,NaN,2001-09-08,USA,tm1,lg1,pos1,nat1
3,pl301,Natalie Beckman,Portland Thorns FC,nwsl,Forward,45.0,0,0,0,0,...,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.17,NaN,2000-03-25,USA,tm1,lg1,pos1,nat1
4,pl154,Ryan Gareis,Houston Dash,nwsl,Forward,21.0,0,0,0,0,...,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.58,NaN,1998-11-13,USA,tm2,lg1,pos1,nat1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,pl268,Rebecca Holloway,Racing Louisville FC,nwsl,Midfielder,25.0,0,1,2,0,...,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.50,NaN,1995-08-25,Northern Ireland,tm3,lg1,pos2,nat17
320,pl74,Jordan Thompson,Washington Spirit,nwsl,Midfielder,37.0,0,0,0,0,...,0,NaN,5.75,141,1997-01-03,Northern Ireland,tm9,lg1,pos2,nat17
321,pl148,Jessica Fishlock,OL Reign,nwsl,Midfielder,10.0,3,0,2,0,...,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.17,119,1987-01-14,Wales,tm7,lg1,pos2,nat23
322,pl302,Jillienne Aguilera,Chicago Red Stars,nwsl,Midfielder,14.0,0,0,0,0,...,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.58,NaN,1998-01-05,Puerto Rico,tm11,lg1,pos2,nat29


In [363]:
# Drop sections no longer needed
player_data_df = player_data_df.drop(["team"], axis=1)
player_data_df = player_data_df.drop(["league"], axis=1)
player_data_df = player_data_df.drop(["position"], axis=1)
player_data_df = player_data_df.drop(["nationality"], axis=1)

In [364]:
# Reorder dataframe
player_data_df = player_data_df[[
                                "player_id",
                                "player_name",
                                "team_id",
                                "league_id",
                                "position_id",
                                "jersey_number",
                                "goals",
                                "assists",
                                "yellow_cards",
                                "red_cards",
                                "shots",
                                "shots_on_goal",
                                "saves",
                                "clean_sheets",
                                "goals_against",
                                "photo_url",
                                "height_(ft)",
                                "weight_(lbs)",
                                "birthdate",
                                "nationality_id"
                            ]]

In [320]:
# Check data types
player_data_df.dtypes

player_id                 object
player_name               object
team_id                   object
league_id                 object
position_id               object
jersey_number            float64
goals                      int64
assists                    int64
yellow_cards               int64
red_cards                  int64
shots                      int64
shots_on_goal              int64
saves                      int64
clean_sheets               int64
goals_against              int64
photo_url                 object
height_(ft)              float64
weight_(lbs)              object
birthdate         datetime64[ns]
nationality_id            object
dtype: object

In [365]:
# Set data type for jersey numbers
jersey_int=[]
for jersey_number in player_data_df["jersey_number"]:
    
    try:
        new_type = int(jersey_number)
    except:
       new_type = None
    
    jersey_int.append(new_type)
    
print(jersey_int)

[9, 33, 24, 45, 21, 39, 5, 14, 17, 13, 22, 77, 25, 5, 17, 13, 15, 3, 7, 11, 25, 28, 23, 15, 24, 23, 34, 8, 10, 23, 30, None, 33, 20, 9, 10, 24, 2, 2, 7, 3, 9, 8, 24, 15, 55, 25, 34, 11, 5, 20, 14, 4, 25, 20, 29, 39, 5, 25, 4, 23, 5, 37, 3, 31, 36, 16, 15, 3, 22, 7, 20, 19, 26, 8, 23, 24, 18, 20, 18, 26, 2, 46, 16, 4, None, 2, 5, 6, 25, 26, 3, 32, 2, 4, 3, 21, 20, 22, 11, 25, 17, 11, 5, 2, 20, 21, 14, 22, 12, 3, 30, 22, 6, 14, 16, 99, 11, 36, 3, 28, 23, 12, 31, 6, 3, 21, 7, 23, 12, 2, 13, 19, 46, 15, 44, 22, 13, 17, 10, 17, 14, 29, 30, 26, 21, 7, 6, 9, 2, 19, None, 2, 10, 4, 14, 16, 6, 28, 24, 14, 22, 12, 6, 8, 23, 11, 8, 14, 20, 15, 16, 13, 33, 12, 7, 6, 18, 7, 17, 19, 13, 32, 12, 13, 19, 16, 23, 26, 39, 9, 8, 17, 28, 24, 44, 21, 13, 32, 4, 16, 2, 29, 26, 8, 19, 28, 25, 18, 1, None, 1, 35, 24, 1, 18, 99, 38, 21, 30, 17, 31, 40, 30, 18, 4, 1, 24, 28, 1, 18, 1, 19, 21, 1, 1, 44, None, 12, 8, 29, 9, 2, 14, 22, 18, 13, 19, 5, 99, 1, 7, 11, 15, 2, 15, None, 10, 17, 22, None, 11, 29, 25, 22,

In [366]:
# Set data type for weight
weight_int=[]
for weight in player_data_df["weight_(lbs)"]:
    
    try:
        new_type = int(weight)
    except:
       new_type = None 
    
    weight_int.append(new_type)
    
print(weight_int)

[121, None, None, None, None, None, None, None, None, None, None, None, None, 128, None, 137, None, None, None, None, None, None, None, 130, None, None, None, None, 126, None, None, None, None, None, None, 117, None, 128, None, None, None, 117, None, None, None, None, None, None, None, None, None, None, 139, 126, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 128, 163, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 126, None, None, None, None, None, None, None, None, 137, 150, None, None, None, None, 139, None, None, 130, 128, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 130, None, None, None, None, None, None, None, None, None, 121, None, None, 117, None, None, None, None, None, None, None, None, None, None, None, None, None, 159, None, None, None, None, None, 121, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, No

In [367]:
# Replace converted sections
player_data_df["jersey_number"] = jersey_int
player_data_df["weight_(lbs)"] = weight_int

In [368]:
# Display Player Data dataframe
player_data_df.head(10)

,player_id,player_name,team_id,league_id,position_id,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,clean_sheets,goals_against,photo_url,height_(ft),weight_(lbs),birthdate,nationality_id
0,pl1,Sophia Smith,tm1,lg1,pos1,9.0,11,5,2,0,75,39,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.50,121.0,2000-08-10,nat1
1,pl157,Hannah Betfort,tm1,lg1,pos1,33.0,2,0,0,0,12,5,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.83,NaN,1999-01-04,nat1
2,pl267,Izzy D'Aquila,tm1,lg1,pos1,24.0,0,0,0,0,16,4,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.58,NaN,2001-09-08,nat1
3,pl301,Natalie Beckman,tm1,lg1,pos1,45.0,0,0,0,0,0,0,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.17,NaN,2000-03-25,nat1
4,pl154,Ryan Gareis,tm2,lg1,pos1,21.0,0,0,0,0,5,1,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.58,NaN,1998-11-13,nat1
5,pl203,Cameron Tucker,tm2,lg1,pos1,39.0,0,0,1,0,7,1,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.75,NaN,1999-07-15,nat1
6,pl94,Paige Monaghan,tm3,lg1,pos1,5.0,3,0,1,0,18,11,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.58,NaN,1996-11-13,nat1
7,pl138,Jessica McDonald,tm3,lg1,pos1,14.0,0,0,0,0,4,1,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,6.00,NaN,1988-02-28,nat1
8,pl167,Maddie Pokorny,tm3,lg1,pos1,17.0,0,1,0,0,5,2,0,0,0,NaN,NaN,NaN,1996-11-24,nat1
9,pl173,Emina Ekic,tm3,lg1,pos1,13.0,0,0,0,0,1,0,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.33,NaN,1999-06-06,nat1


### Exports:
---

In [ ]:
# Export CSV 1-Player Data

In [ ]:
# Export CSV 2-Teams Data

In [ ]:
# Export CSV 3-Leagues Data

In [ ]:
# Export CSV 4-Positions Data

In [ ]:
# Export CSV 5-Nationalities Data